# Basic settings

In [1]:
(ext:chdir "~/Development/clasp/extensions/cando/src/tests/boron/" t)

0

#### Create all paths.

In [2]:
(leap.core::clear-path)
(add-path "~/Development/clasp/extensions/cando/src/tests/boron/")
(add-path "~/Development/amber16/dat/leap/cmd/")
(add-path "~/Development/amber16/dat/leap/lib/")
(add-path "~/Development/amber16/dat/leap/parm/")
(add-path "~/Development/amber16/dat/leap/prep/")
(add-path "~/Development/amber16/dat/antechamber/")

(#P"/Users/tuj24515/Development/amber16/dat/antechamber/"
 #P"/Users/tuj24515/Development/amber16/dat/leap/prep/"
 #P"/Users/tuj24515/Development/amber16/dat/leap/parm/"
 #P"/Users/tuj24515/Development/amber16/dat/leap/lib/"
 #P"/Users/tuj24515/Development/amber16/dat/leap/cmd/"
 #P"/Users/tuj24515/Development/clasp/extensions/cando/src/tests/boron/")

In [3]:
(start-swank)

swank-loader -> /Users/tuj24515/Development/slime/swank-loader.lisp

"Read: (DEFPACKAGE :SWANK-LOADER (:USE :CL) (:EXPORT :INIT :DUMP-IMAGE :LIST-FASLS :*SOURCE-DIRECTORY* :*FASL-DIRECTORY*))
" 
"Read: (IN-PACKAGE :SWANK-LOADER)
" 
"Read: (DEFVAR *SOURCE-DIRECTORY* (MAKE-PATHNAME :NAME NIL :TYPE NIL :DEFAULTS (OR *LOAD-PATHNAME* *DEFAULT-PATHNAME-DEFAULTS*)) The directory where to look for the source.)
" 
"Read: (DEFPARAMETER *SYSDEP-FILES* (QUOTE ((SWANK CLASP) (SWANK GRAY))))
" 
"Read: (DEFPARAMETER *IMPLEMENTATION-FEATURES* (QUOTE (:ALLEGRO :LISPWORKS :SBCL :CLOZURE :CMU :CLISP :CCL :CORMAN :CORMANLISP :ARMEDBEAR :GCL :ECL :SCL :MKCL :CLASP)))
" 
"Read: (DEFPARAMETER *OS-FEATURES* (QUOTE (:MACOSX :LINUX :WINDOWS :MSWINDOWS :WIN32 :SOLARIS :DARWIN :SUNOS :HPUX :UNIX)))
" 
"Read: (DEFPARAMETER *ARCHITECTURE-FEATURES* (QUOTE (:POWERPC :PPC :X86 :X86-64 :X86_64 :AMD64 :I686 :I586 :I486 :PC386 :IAPX386 :SPARC64 :SPARC :HPPA64 :HPPA :ARM :ARMV5L :ARMV6L :ARMV7L :ARM64 :PENTIUM3 :PENTIUM4 

T

#### Load the GAFF force field and boronic ester frcmod file.

In [78]:
(clear-force-field)
(load-atom-type-rules "ATOMTYPE_GFF.DEF")

(#<FORCE-FIELD ATOMTYPE_GFF.DEF>)

In [79]:
(load-amber-params "gaff.dat")

SIMPLE-WARNING: 
Warning!  Skipping 10-12 hbond
SIMPLE-WARNING: 
Warning!  Skipping force field atom equivalences


#<FORCE-FIELD gaff.dat>

In [80]:
(load-atom-type-rules "boron.def")
(load-atom-type-rules "water_gaff.def")

(#<FORCE-FIELD water_gaff.def> #<FORCE-FIELD boron.def> #<FORCE-FIELD gaff.dat>
 #<FORCE-FIELD ATOMTYPE_GFF.DEF>)

In [81]:
(load-amber-params "frcmod.boronic-ester")
(gethash :default leap.core::*force-fields*)

(#<FORCE-FIELD frcmod.boronic-ester> #<FORCE-FIELD water_gaff.def>
 #<FORCE-FIELD boron.def> #<FORCE-FIELD gaff.dat>
 #<FORCE-FIELD ATOMTYPE_GFF.DEF>)

#### Merge force field and define parameter ff.

In [82]:
(:= *ff* (leap.core::merged-force-field))

SIMPLE-WARNING: 
Implement merge of FFVdwDb
SIMPLE-WARNING: 
Implement merge of FFVdwDb
SIMPLE-WARNING: 
Implement merge of FFVdwDb
SIMPLE-WARNING: 
Implement merge of FFVdwDb
SIMPLE-WARNING: 
Implement merge of FFVdwDb


*FF*

## Put sugar to boronic ester.

#### Load boronic ester file.

In [160]:
(defparameter *b03agg* (cando:load-chem-draw-aggregate "boronic-ester03.cdxml"))

*B03AGG*

In [161]:
(progn
  (defparameter *stereocenters*
    (sort (cando:gather-stereocenters *b03agg*) #'string< :key #'chem:get-name))
  (cando:set-stereoisomer-func *stereocenters* (constantly :S) :show t)
  (let ((quat-matcher (core:make-cxx-object 'chem:chem-info)))
    (chem:compile-smarts quat-matcher "[C&H0&D4]")
    (chem:map-atoms nil (lambda (a) (when (chem:matches quat-matcher a)
                  (chem:set-configuration a :S)
                  (format t "Set atom ~a to :S~%" (chem:get-name a))))
            *b03agg*)))

C_133  S
C_135  S
C_208  S
C_210  S
C_233  S
C_235  S
C_2572 S
C_2574 S
C_2582 S
C_2584 S
C_2592 S
C_2594 S
Set atom C_2584 to :S
Set atom C_2574 to :S
Set atom C_2594 to :S
Set atom C_235 to :S
Set atom C_210 to :S
Set atom C_135 to :S


NIL

#### Assign types and minimize.

In [162]:
(chem:assign-types *ff* *b03agg*)

NIL

In [163]:
(jostle *b03agg*)

NIL

In [164]:
(defparameter *b03e* (energy:minimize *b03agg* :force-field *ff* :restraints-on t))

======= Starting Steepest Descent Minimizer
---Stage-Seconds--Step-----Alpha---Dir-------------Energy-----------RMSforce
 minSDnP       0     1  0.000000   0.0              0.000         280001.012
 minSDnP       0     2  0.000397   0.0     4466428308.273         124964.700
 minSDnP       0     3  0.000436   0.0     1913853829.598          81420.166
 minSDnP       0     4  0.000363   0.0     1010375930.502          51250.303
 minSDnP       0     5  0.000405   0.0      612182500.187          40302.805
 minSDnP       0     6  0.000345   0.0      402355411.734          29017.094
 minSDnP       0     7  0.000388   0.0      280175052.192          24792.870
 minSDnP       0     8  0.000339   0.0      202129738.138          19043.651
 minSDnP       0     9  0.000386   0.0      150711659.758          16845.914
 minSDnP       0    10  0.000358   0.0      112802783.152          13535.933
---Stage-Seconds--Step-----Alpha---Dir-------------Energy-----------RMSforce
 minSDnP       0    11  0.000382

SIMPLE-WARNING: 
Estimated angle term :ne-:c2-:ne  T0 -> 109.500000 degrees K -> 70.000000 kCal/rad^2
SIMPLE-WARNING: 
Estimated angle term :c2-:c2-:ne  T0 -> 109.500000 degrees K -> 70.000000 kCal/rad^2


*B03E*

In [165]:
(nglv::show-aggregate *b03agg*)

A Jupyter Widget

#### Load sugar file.

####  Assign types and minimize.

#### Remove ome residues from boronic ester.

In [166]:
(:= *m* (chem:content-at *b03agg* 0))
(:= *res* (chem:content-with-name *m* :ome1))
(chem:map-atoms nil (lambda (a) (format t "Residue: ~a~%" (chem:get-name a))) *res*)

Residue: O
Residue: C_2781
Residue: H_27811
Residue: H_27812
Residue: H_27813


NIL

In [167]:
(:= *atom* (chem:content-with-name *res* :H_27813))
(chem:remove-atom-delete-bonds *res* *atom*)
(:= *atom* (chem:content-with-name *res* :H_27812))
(chem:remove-atom-delete-bonds *res* *atom*)
(:= *atom* (chem:content-with-name *res* :H_27811))
(chem:remove-atom-delete-bonds *res* *atom*)
(:= *atom* (chem:content-with-name *res* :C_2781))
(chem:remove-atom-delete-bonds *res* *atom*)
(:= *atom* (chem:content-with-name *res* :O))
(chem:remove-atom-delete-bonds *res* *atom*)

NIL

In [168]:
(chem:remove-residue *m* *res*)

NIL

In [128]:
(:= *res* (chem:content-with-name *m* :ome2))
(chem:map-atoms nil (lambda (a) (format t "Residue: ~a~%" (chem:get-name a))) *res*)

Residue: C_2783
Residue: O
Residue: H_27831
Residue: H_27832
Residue: H_27833


NIL

In [129]:
(:= *atom* (chem:content-with-name *res* :H_27833))
(chem:remove-atom-delete-bonds *res* *atom*)
(:= *atom* (chem:content-with-name *res* :H_27832))
(chem:remove-atom-delete-bonds *res* *atom*)
(:= *atom* (chem:content-with-name *res* :H_27831))
(chem:remove-atom-delete-bonds *res* *atom*)
(:= *atom* (chem:content-with-name *res* :C_2783))
(chem:remove-atom-delete-bonds *res* *atom*)
(:= *atom* (chem:content-with-name *res* :O))
(chem:remove-atom-delete-bonds *res* *atom*)
(chem:remove-residue *m* *res*)

NIL

In [130]:
(:= *res* (chem:content-with-name *m* :ome3))
(chem:map-atoms nil (lambda (a) (format t "Residue: ~a~%" (chem:get-name a))) *res*)

Residue: O
Residue: C_2981
Residue: H_29811
Residue: H_29812
Residue: H_29813


NIL

In [131]:
(:= *atom* (chem:content-with-name *res* :H_29813))
(chem:remove-atom-delete-bonds *res* *atom*)
(:= *atom* (chem:content-with-name *res* :H_29812))
(chem:remove-atom-delete-bonds *res* *atom*)
(:= *atom* (chem:content-with-name *res* :H_29811))
(chem:remove-atom-delete-bonds *res* *atom*)
(:= *atom* (chem:content-with-name *res* :C_2981))
(chem:remove-atom-delete-bonds *res* *atom*)
(:= *atom* (chem:content-with-name *res* :O))
(chem:remove-atom-delete-bonds *res* *atom*)
(chem:remove-residue *m* *res*)

NIL

In [132]:
(:= *res* (chem:content-with-name *m* :ome4))
(chem:map-atoms nil (lambda (a) (format t "Residue: ~a~%" (chem:get-name a))) *res*)

Residue: C_2739
Residue: O
Residue: H_27391
Residue: H_27392
Residue: H_27393


NIL

In [133]:
(:= *atom* (chem:content-with-name *res* :H_27393))
(chem:remove-atom-delete-bonds *res* *atom*)
(:= *atom* (chem:content-with-name *res* :H_27392))
(chem:remove-atom-delete-bonds *res* *atom*)
(:= *atom* (chem:content-with-name *res* :H_27391))
(chem:remove-atom-delete-bonds *res* *atom*)
(:= *atom* (chem:content-with-name *res* :C_2739))
(chem:remove-atom-delete-bonds *res* *atom*)
(:= *atom* (chem:content-with-name *res* :O))
(chem:remove-atom-delete-bonds *res* *atom*)
(chem:remove-residue *m* *res*)

NIL

In [23]:
(:= *res* (chem:content-with-name *m* :ome5))
(chem:map-atoms nil (lambda (a) (format t "Residue: ~a~%" (chem:get-name a))) *res*)

Residue: O
Residue: C_2866
Residue: H_28661
Residue: H_28662
Residue: H_28663


NIL

In [24]:
(:= *atom* (chem:content-with-name *res* :H_28663))
(chem:remove-atom-delete-bonds *res* *atom*)
(:= *atom* (chem:content-with-name *res* :H_28662))
(chem:remove-atom-delete-bonds *res* *atom*)
(:= *atom* (chem:content-with-name *res* :H_28661))
(chem:remove-atom-delete-bonds *res* *atom*)
(:= *atom* (chem:content-with-name *res* :C_2866))
(chem:remove-atom-delete-bonds *res* *atom*)
(:= *atom* (chem:content-with-name *res* :O))
(chem:remove-atom-delete-bonds *res* *atom*)
(chem:remove-residue *m* *res*)

NIL

In [169]:
(nglv::show-aggregate *b03agg*)

A Jupyter Widget

In [170]:
(:= *tnagg* (load-mol2 "trinag.mol2"))
(:= *tnmol* (chem:content-at *tnagg* 0))

*TNMOL*

In [171]:
(:= v (nglv:show-aggregate *tnagg*))
v

A Jupyter Widget

In [172]:
(:= *b03mol* (chem:matter-copy (chem:content-at *b03agg* 0)))

*B03MOL*

In [173]:
(:= *ta* (chem:make-aggregate))
(chem:add-matter *ta* *b03mol*)
(chem:add-matter *ta* *tnmol*)

NIL

In [174]:
(:= v (nglv:show-aggregate *ta*))
v

A Jupyter Widget

In [175]:
(:= *m* (chem:content-at *ta* 0))
(:= *bor1* (chem:content-with-name *m* :bor1))
(:= *b1* (chem:content-with-name *bor1* :B1))

*B1*

In [141]:
(:= *m* (chem:content-at *ta* 0))
(:= *bor2* (chem:content-with-name *m* :bor2))
(:= *b2* (chem:content-with-name *bor2* :B2))

*B2*

In [142]:
(:= *bor3* (chem:content-with-name *m* :bor3))
(:= *b3* (chem:content-with-name *bor3* :B3))

*B3*

In [143]:
(:= *bor4* (chem:content-with-name *m* :bor4))
(:= *b4* (chem:content-with-name *bor4* :B4))

*B4*

In [176]:
(:= *m* (chem:content-at *ta* 1))
(chem:map-residues 'list (lambda (a) (chem:get-name a)) *m*)

(:AMU :NAG :AMU)

In [177]:
(:= *amu1* (chem:content-at *m* 0))

*AMU1*

In [178]:
(chem:map-atoms 'list (lambda (a) (chem:get-name a)) *amu1*)

(:C1 :C2 :C3 :C4 :C5 :C6 :C7 :C8 :C9 :C10 :C11 :O3 :O4 :O5 :O6 :O7 :O10 :N2)

In [179]:
(:= *amu1o6* (chem:content-with-name *amu1* :o6))

*AMU1O6*

In [148]:
(:= *nag* (chem:content-at *m* 1))
(:= *nago3* (chem:content-with-name *nag* :o3))

*NAGO3*

In [149]:
(:= *nag* (chem:content-at *m* 1))
(:= *nago6* (chem:content-with-name *nag* :o6))

*NAGO6*

In [150]:
(:= *amu3* (chem:content-at *m* 2))
(:= *amu3o6* (chem:content-with-name *amu3* :o6))

*AMU3O6*

In [151]:
(chem:bond-to *amu3o6* *b3* :single-bond)

(Bond #<ATOM :O6/:O>-#<ATOM :B3/:B> @0x1283baec8)

In [152]:
(chem:bond-to *nago6* *b4* :single-bond)

(Bond #<ATOM :O6/:O>-#<ATOM :B4/:B> @0x127d4c4c8)

In [153]:
(chem:bond-to *nago3* *b2* :single-bond)

(Bond #<ATOM :O3/:O>-#<ATOM :B2/:B> @0x127ecf788)

In [180]:
(chem:bond-to *amu1o6* *b1* :single-bond)

(Bond #<ATOM :O6/:O>-#<ATOM :B1/:B> @0x12783f488)

In [181]:
(chem:assign-types *ff* *ta*)

NIL

In [182]:
(jostle *ta*)

NIL

In [183]:
(energy:minimize *ta* :force-field *ff* :restraints-on t)

SIMPLE-PROGRAM-ERROR: 
Could not find atom: #<ATOM :O6/:O> in atom-vertex map in RingFinder


NIL

In [159]:
(:= v (nglv:show-aggregate *ta*))
v

A Jupyter Widget

In [58]:
(geta :O6)

#<ATOM :O6/:O>

In [57]:
(defun geta (name) (chem:atom-with-name (chem:content-at (chem:content-at *tnagg* 0) 1) name))

GETA

In [37]:
(:= *twister* (core:make-cxx-object 'chem:twister))

*TWISTER*

In [41]:
(chem:define-fixed-and-mobile *twister* (geta :c5) (geta :c6))

NIL

In [42]:
(chem:rotate *twister* (* 120.0 0.0174533))

NIL

In [43]:
(:= v (nglv:show-aggregate *tnagg*))
v

A Jupyter Widget

In [44]:
(/ (apply 'geom:calculate-dihedral (mapcar (lambda (a) (chem:get-position (geta a))) '(:o5 :c5 :c6 :o6))) 0.01745)

58.83488443026842d0

In [58]:
(:= *b02mol* (chem:matter-copy (chem:content-at *b02agg* 0)))

*B02MOL*

In [88]:
(chem:map-residues nil 
                   (lambda (r) 
                           (chem:add-matter *b02mol* r))
                   *tnmol*)

NIL

In [89]:
(:= *newagg* (chem:make-aggregate))
(chem:add-matter *newagg* *b02mol*)

NIL

In [90]:
(nglv:show-aggregate *newagg*)

A Jupyter Widget

#### Connect sugar and boronic ester. 

In [39]:
(:= *m* (chem:content-at *b02agg* 0))
(:= *bor1* (chem:content-with-name *m* :bor1))
(:= *b1* (chem:content-with-name *bor1* :B1))

*B1*

In [40]:
(:= *bor2* (chem:content-with-name *m* :bor2))
(:= *b2* (chem:content-with-name *bor2* :B2))

*B2*

In [41]:
(:= *bor3* (chem:content-with-name *m* :bor3))
(:= *b3* (chem:content-with-name *bor3* :B3))

*B3*

In [42]:
(:= *bor4* (chem:content-with-name *m* :bor4))
(:= *b4* (chem:content-with-name *bor4* :B4))

*B4*

In [44]:
(:= *bor5* (chem:content-with-name *m* :bor5))
(:= *b5* (chem:content-with-name *bor5* :B5))

*B5*

In [ ]:
(chem:map-residues nil (lambda (r) (format t "Residue: ~a~%" (chem:get-name r))) *b02agg*)

In [45]:
(:= *msugar* (chem:content-at *sugaragg* 0))
(:= *rsugar* (chem:content-at *msugar* 0))

*RSUGAR*

In [46]:
(chem:add-matter *m* *rsugar*)

NIL

In [50]:
(:= *sugar* (chem:content-at *msugar* 0))

*SUGAR*

In [51]:
(chem:map-atoms 'nil (lambda (a) (chem:get-name a)) *sugar*)

NIL

In [48]:
(:= *so1* (chem:content-at *rsugar* :SO1))

1An error occurred of type: TYPE-ERROR: 
  #<TYPE-ERROR>


NIL

In [186]:
(:= *h1* (chem:content-with-name *rmoe* :H1))

*H1*

In [187]:
(chem:remove-atom-delete-bonds *rmoe* *h1*)

NIL

In [188]:
(chem:bond-to *o1* *b* :single-bond)

(Bond #<ATOM :O1/:O>-#<ATOM :B1/:B> @0x1162253c8)

In [189]:
(defparameter *agg1* (chem:make-aggregate))

*AGG1*

In [190]:
(chem:add-matter *agg1* *m*)

NIL

In [191]:
(nglv::show-aggregate *agg1*)

A Jupyter Widget

In [192]:
(chem:assign-types *ff* *agg1*)
(jostle *agg1*)
(energy:minimize *agg1* :force-field *ff* :restraints-on t)

======= Starting Steepest Descent Minimizer
---Stage-Seconds--Step-----Alpha---Dir-------------Energy-----------RMSforce
 minSDnP       0     1  0.000000   0.0              0.000          16130.034
 minSDnP       0     2  0.000311   0.0       17903796.195          15411.899
 minSDnP       0     3  0.000056   0.0       16045987.939           6440.800
 minSDnP       0     4  0.000298   0.0       12367254.323          11004.390
DONE absolute force test:
forceRmsMag(3783.217480).LT.forceTolerance(5000.000000)
======= Starting Conjugate Gradient Minimizer
 minCGnP       0     5  0.000000   0.0              0.000           3783.217
 minCGnP       0     6  0.000349  69.9        9837765.956          10356.243
 minCGnP       0     7  0.000010  33.5        9481771.473           8072.692
 minCGnP       0     8  0.000029  35.5        8988623.146           4948.602
 minCGnP       0     9  0.000118  54.2        8573060.932           5856.798
 minCGnP       0    10  0.000038  32.2        8355165.749 

SIMPLE-WARNING: 
Estimated angle term :ne-:c2-:ne  T0 -> 109.500000 degrees K -> 70.000000 kCal/rad^2
SIMPLE-WARNING: 
Estimated angle term :c2-:c2-:ne  T0 -> 109.500000 degrees K -> 70.000000 kCal/rad^2


#<ENERGY-FUNCTION >

In [193]:
(nglv::show-aggregate *agg1*)

A Jupyter Widget

In [77]:
(chem:map-residues nil (lambda (r) (format t "Residue: ~a~%" (chem:get-name r))) *b02agg*)

Residue: BOR5
Residue: NIL
Residue: NIL
Residue: NIL
Residue: BOR3
Residue: BOR4
Residue: NIL
Residue: NIL
Residue: NIL
Residue: NIL
Residue: BOR2
Residue: BOR1
Residue: NIL
Residue: NIL
Residue: OME2
Residue: OME5
Residue: OME4
Residue: OME3
Residue: NIL
Residue: NIL
Residue: NIL


NIL

In [40]:
(:= *m* (chem:content-at *b02agg* 0))

*M*

In [42]:
(chem:remove-residue *m* (chem:content-with-name *m* :ome2))

NIL

In [43]:
(chem:atom-with-name *m* :b1)

SIMPLE-PROGRAM-ERROR: 
There are no applicable methods of ATOM-WITH-NAME for receiver class MOLECULE


NIL

#### Save the topology file and coordinate file.

In [44]:
(chem:set-property *b01agg* :bounding-box '(30.0 30.0 30.0))

NIL

In [45]:
(chem:assign-types *ff* *b01agg*)

NIL

In [46]:
(leap.topology:save-amber-parm-format *b01agg* "b01.top" "b01.crd" *ff* :assign-types nil)

atom1: #<ATOM :C_312/:C> atom2: #<ATOM :C_310/:C> i1: 0 i2: 3
 atom1: #<ATOM :C_312/:C> atom2: #<ATOM :H_312/:H> i1: 0 i2: 588
 atom1: #<ATOM :C_310/:C> atom2: #<ATOM :C_308/:C> i1: 3 i2: 9
 atom1: #<ATOM :C_310/:C> atom2: #<ATOM :B/:B> i1: 3 i2: 12
 atom1: #<ATOM :C_314/:C> atom2: #<ATOM :C_312/:C> i1: 6 i2: 0
 atom1: #<ATOM :C_314/:C> atom2: #<ATOM :C_298/:C> i1: 6 i2: 15
 atom1: #<ATOM :C_314/:C> atom2: #<ATOM :H_314/:H> i1: 6 i2: 585
 atom1: #<ATOM :C_308/:C> atom2: #<ATOM :C_306/:C> i1: 9 i2: 18
 atom1: #<ATOM :C_308/:C> atom2: #<ATOM :H_308/:H> i1: 9 i2: 582
 atom1: #<ATOM :B/:B> atom2: #<ATOM :OH/:O> i1: 12 i2: 24
 atom1: #<ATOM :C_298/:C> atom2: #<ATOM :C_281/:C> i1: 15 i2: 30
 atom1: #<ATOM :C_306/:C> atom2: #<ATOM :C_298/:C> i1: 18 i2: 15
 atom1: #<ATOM :C_306/:C> atom2: #<ATOM :H_306/:H> i1: 18 i2: 579
 atom1: #<ATOM :OH/:O> atom2: #<ATOM :B/:B> i1: 21 i2: 12
 atom1: #<ATOM :OH/:O> atom2: #<ATOM :HH/:H> i1: 21 i2: 639
 atom1: #<ATOM :OH/:O> atom2: #<ATOM :HH/:H> i1: 24 i2: 6

0

#### Calculate each energy parameters.

In [47]:
(chem:get-energy (chem:get-stretch-component *b01e*))

82.36554103619612d0

In [48]:
(chem:get-energy (chem:get-angle-component *b01e*))

105.9430062847246d0

In [49]:
(chem:get-energy (chem:get-dihedral-component *b01e*))

189.36024506338896d0

In [50]:
(chem:get-energy (chem:get-nonbond-component *b01e*))

240.12762920122927d0

#### Create Tip3 water molecules box.

In [51]:
(load-off "tip3pfbox.off")

(:TIP3PFBOX)

In [52]:
(nglv::show-aggregate ~tip3pfbox)

A Jupyter Widget

In [53]:
(print ~tip3pfbox)


AGGREGATE 

AGGREGATE

In [54]:
(defparameter *solvent* ~tip3pfbox)

*SOLVENT*

In [55]:
(chem:assign-types *ff* *solvent*)

NIL

#### Add Tip3 water molecules to the boronic ester.

In [56]:
(:= *b01wagg* (leap.solvate::tool-solvate-and-shell *b01agg* *solvent* '(30.0 30.0 30.0) :shell t :farness 7.0))

Total bounding box for atom centers:  81.73844693175073d0 85.77759373786844d0 66.99863433109397d0


*B01WAGG*

In [57]:
(chem:map-atoms 'list (lambda (a) (chem:get-type a)) *b01wagg*)

(:|c2| :|c2| :|c2| :|c2| :|b| :|c2| :|c2| :|oh| :|oh| :|os| :|c3| :|c3| :|n|
 :|c3| :|h1| :|c3| :|c3| :|c| :|c3| :|h1| :|oh| :|c3| :|h1| :|oh| :|c3| :|c3|
 :|c| :|c3| :|o| :|os| :|h1| :|c3| :|h2| :|os| :|c3| :|n3| :|o| :|n| :|c3|
 :|oh| :|c3| :|c| :|c3| :|c3| :|c3| :|c3| :|h1| :|o| :|oh| :|n| :|c| :|os|
 :|h1| :|c3| :|os| :|h1| :|c3| :|c| :|c3| :|o| :|n| :|b| :|c3| :|h1| :|oh|
 :|oh| :|c3| :|o| :|c2| :|c3| :|c2| :|oh| :|oh| :|h2| :|os| :|c3| :|c2| :|c2|
 :|c3| :|c2| :|c2| :|c3| :|n| :|c| :|c3| :|c3| :|h1| :|c| :|c3| :|o| :|n| :|os|
 :|h1| :|c3| :|os| :|h1| :|c3| :|c3| :|o| :|c2| :|c3| :|b| :|c3| :|h1| :|oh|
 :|oh| :|c3| :|c2| :|c2| :|c3| :|c2| :|oh| :|oh| :|h2| :|os| :|c2| :|c2| :|n|
 :|c| :|b| :|c3| :|c| :|n| :|o| :|c2| :|oh| :|oh| :|c2| :|o| :|c3| :|c2| :|c2|
 :|c2| :|c2| :|c2| :|c2| :|c2| :|c2| :|c2| :|c2| :|ha| :|ha| :|ha| :|ha| :|ha|
 :|ha| :|ha| :|ha| :|ha| :|h1| :|h1| :|h1| :|h1| :|ha| :|ha| :|ha| :|ha| :|hc|
 :|hc| :|h1| :|h1| :|h1| :|h1| :|h1| :|h1| :|h1| :|ha| :|ha| :|ha| :|h

#### Minimize the energy of the water and boronic ester.

In [58]:
(defparameter *b01we* (energy:minimize *b01wagg* :force-field *ff* :restraints-on t))

======= Starting Steepest Descent Minimizer
---Stage-Seconds--Step-----Alpha---Dir-------------Energy-----------RMSforce
 minSDnP       0     1  0.000000   0.0              0.000           7248.333
DONE absolute force test:
forceRmsMag(137.808063).LT.forceTolerance(5000.000000)
======= Starting Conjugate Gradient Minimizer
 minCGnP       0     2  0.000000   0.0              0.000            137.808
 minCGnP       0     3  0.000025  35.9           8188.222             99.704
 minCGnP       0     4  0.000127  70.9           5681.766            244.675
 minCGnP       0     5  0.000034  25.1           2823.308             50.961
 minCGnP       0     6  0.000392  12.4           1633.479             20.881
 minCGnP       0     7  0.000103  61.6           1531.649             38.159
 minCGnP       0     8  0.000037  65.4           1434.335             52.674
 minCGnP       0     9  0.000070  56.1           1175.590             13.503
 minCGnP       0    10  0.000310  70.8           1069.034  

*B01WE*

In [59]:
(nglv::show-aggregate *b01wagg*)

A Jupyter Widget

#### Save the topology file and coordinate file.

In [60]:
(chem:assign-types *ff* *b01wagg*)

NIL

In [61]:
(leap.topology:save-amber-parm-format *b01wagg* "b01_water.top" "b01_water.crd" *ff* :assign-types nil)

atom1: #<ATOM :C_312/:C> atom2: #<ATOM :C_310/:C> i1: 0 i2: 3
 atom1: #<ATOM :C_312/:C> atom2: #<ATOM :C_314/:C> i1: 0 i2: 6
 atom1: #<ATOM :C_308/:C> atom2: #<ATOM :C_310/:C> i1: 9 i2: 3
 atom1: #<ATOM :B/:B> atom2: #<ATOM :C_310/:C> i1: 12 i2: 3
 atom1: #<ATOM :C_298/:C> atom2: #<ATOM :C_314/:C> i1: 15 i2: 6
 atom1: #<ATOM :C_306/:C> atom2: #<ATOM :C_308/:C> i1: 18 i2: 9
 atom1: #<ATOM :C_306/:C> atom2: #<ATOM :C_298/:C> i1: 18 i2: 15
 atom1: #<ATOM :OH/:O> atom2: #<ATOM :B/:B> i1: 21 i2: 12
 atom1: #<ATOM :OH/:O> atom2: #<ATOM :B/:B> i1: 24 i2: 12
 atom1: #<ATOM :O/:O> atom2: #<ATOM :B/:B> i1: 27 i2: 12
 atom1: #<ATOM :C_281/:C> atom2: #<ATOM :C_298/:C> i1: 30 i2: 15
 atom1: #<ATOM :C_724/:C> atom2: #<ATOM :O/:O> i1: 33 i2: 27
 atom1: #<ATOM :C_724/:C> atom2: #<ATOM :H/:H> i1: 33 i2: 42
 atom1: #<ATOM :C_724/:C> atom2: #<ATOM :C_723/:C> i1: 33 i2: 45
 atom1: #<ATOM :N/:N> atom2: #<ATOM :C_110/:C> i1: 36 i2: 48
 atom1: #<ATOM :N/:N> atom2: #<ATOM :C_281/:C> i1: 36 i2: 30
 atom1: #<AT

0

In [62]:
(chem:get-energy (chem:get-stretch-component *b01we*))

83.20676890860902d0

In [63]:
(chem:get-energy (chem:get-angle-component *b01we*))

109.30096341569147d0

In [64]:
(chem:get-energy (chem:get-dihedral-component *b01we*))

188.36619959633745d0

In [65]:
(chem:get-energy (chem:get-nonbond-component *b01we*))

-214.9212268510093d0

In [14]:
(chem:map-residues nil (lambda (r) (print r)) ~tip3pfbox)


#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WA

NIL

In [47]:
(chem:map-atoms nil (lambda (r) (print r)) *solvent*)


#<ATOM :O/:O> 
#<ATOM :H1/:H> 
#<ATOM :H2/:H> 
#<ATOM :O/:O> 
#<ATOM :H1/:H> 
#<ATOM :H2/:H> 
#<ATOM :O/:O> 
#<ATOM :H1/:H> 
#<ATOM :H2/:H> 
#<ATOM :O/:O> 
#<ATOM :H1/:H> 
#<ATOM :H2/:H> 
#<ATOM :O/:O> 
#<ATOM :H1/:H> 
#<ATOM :H2/:H> 
#<ATOM :O/:O> 
#<ATOM :H1/:H> 
#<ATOM :H2/:H> 
#<ATOM :O/:O> 
#<ATOM :H1/:H> 
#<ATOM :H2/:H> 
#<ATOM :O/:O> 
#<ATOM :H1/:H> 
#<ATOM :H2/:H> 
#<ATOM :O/:O> 
#<ATOM :H1/:H> 
#<ATOM :H2/:H> 
#<ATOM :O/:O> 
#<ATOM :H1/:H> 
#<ATOM :H2/:H> 
#<ATOM :O/:O> 
#<ATOM :H1/:H> 
#<ATOM :H2/:H> 
#<ATOM :O/:O> 
#<ATOM :H1/:H> 
#<ATOM :H2/:H> 
#<ATOM :O/:O> 
#<ATOM :H1/:H> 
#<ATOM :H2/:H> 
#<ATOM :O/:O> 
#<ATOM :H1/:H> 
#<ATOM :H2/:H> 
#<ATOM :O/:O> 
#<ATOM :H1/:H> 
#<ATOM :H2/:H> 
#<ATOM :O/:O> 
#<ATOM :H1/:H> 
#<ATOM :H2/:H> 
#<ATOM :O/:O> 
#<ATOM :H1/:H> 
#<ATOM :H2/:H> 
#<ATOM :O/:O> 
#<ATOM :H1/:H> 
#<ATOM :H2/:H> 
#<ATOM :O/:O> 
#<ATOM :H1/:H> 
#<ATOM :H2/:H> 
#<ATOM :O/:O> 
#<ATOM :H1/:H> 
#<ATOM :H2/:H> 
#<ATOM :O/:O> 
#<ATOM :H1/:H> 
#<ATOM :H2/:H> 
#<ATOM :O/:O

NIL

In [29]:
(defparameter *cat* (cando:load-chem-draw-aggregate "spiroligomer transglycosylase copy.cdxml"))

SIMPLE-ERROR: 
Could not find file spiroligomer transglycosylase copy.cdxml


NIL

In [ ]:
(chem:residue-with-name )